In [ ]:
import cv2
from IPython.display import YouTubeVideo, display, HTML
from matplotlib import rcParams
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable

rcParams["animation.embed_limit"] = 2**128

## Image Sharpness Measure for Blurred Images in Frequency Domain

Implementar un detector de máximo enfoque sobre un video aplicando técnicas de análisis espectral similar al que utilizan las cámaras digitales modernas.

1. Se debe implementar un algoritmo que dada una imagen, o región, calcule la métrica propuesta en el paper [*Image Sharpness Measure for Blurred Images in Frequency Domain*](https://www.sciencedirect.com/science/article/pii/S1877705813016007) y realizar tres experimentos.
    1. Medición sobre todo el frame.
    2. Medición sobre una ROI ubicada en el centro del frame. Area de la ROI = 5 o 10% del area total del frame.
    3. Medición sobre una matriz de enfoque compuesta por un arreglo de NxM elementos rectangulares equiespaciados. N y M son valores arbitrarios, probar con varios valores 3x3, 7x5, etc.

    Para cada experimento se debe presentar:
    - Una curva o varias curvas que muestren la evolución de la métrica frame a frame donde se vea claramente cuando el algoritmo detectó el punto de máximo enfoque.
    - Video con la ROI o matriz, graficada en rojo y superpuesta al video original para los frames que no están en foco y en verde para los frames donde se detecta la condición de máximo enfoque.

2. Calcular la métrica de enfoque eligiendo uno de los algoritmos explicados en el apéndice de: [*Analyze of focus measure operators in shapeform focus*](https://www.researchgate.net/publication/234073157_Analysis_of_focus_measure_operators_in_shape-from-focus).

El algoritmo de detección a implementar debe detectar y devolver los puntos de máximo enfoque de manera automática.

Extra: Aplicar unsharp masking para expandir la zona de enfoque y devolver.

In [ ]:
YOUTUBE_VIDEO_ID = "Nn6EJunyOSI"
video = YouTubeVideo(YOUTUBE_VIDEO_ID, width=700, height=438)
display(video)

Cargar video a analizar y previsualización de un frame:

In [ ]:
def get_video_frames(input_video_path: str) -> list[np.ndarray]:
    """Get list of frames from a video given its path"""
    cap = cv2.VideoCapture(input_video_path)
    video_frames = []
    # read until is completed
    while cap.isOpened():
        # capture frame-by-frame
        ret, frame = cap.read()
        if not ret:  # finished
            break
        video_frames.append(frame[..., ::-1])
    cap.release()
    return video_frames

In [ ]:
VIDEO_FILE_PATH = "video/focus_video.mov"
video_frames = get_video_frames(VIDEO_FILE_PATH)

initial_frame_gray = cv2.cvtColor(video_frames[0], cv2.COLOR_RGB2GRAY)

frame_fft = np.fft.fft2(initial_frame_gray)
frame_fft = np.fft.fftshift(frame_fft)  # low freq to origin for visualization
frame_fft = 20 * np.log(np.abs(frame_fft))  # get module

fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(12, 10)
axs[0].set_title("video frame")
axs[0].imshow(video_frames[0])
axs[1].set_title("video frame grayscale")
axs[1].imshow(initial_frame_gray, cmap="gray")
axs[2].set_title("video frame fft")
axs[2].imshow(frame_fft, cmap="jet")

Implementación del algoritmo que calcula la métrica propuesta en el paper [*Image Sharpness Measure for Blurred Images in Frequency Domain*](https://www.sciencedirect.com/science/article/pii/S1877705813016007)

In [ ]:
def get_image_fm_quality(input_img: np.ndarray) -> float:
    """
    Image quality measure (FM)
    Where FM stands for Frequency Domain Image Blur Measure.
    """
    # 1. Compute F which is the Fourier Transform representation of the image I
    img_fft = np.fft.fft2(input_img)
    # 2. Find Fc which is obtained by shifting the origin of F to centre
    img_fft_center = np.fft.fftshift(img_fft)
    # 3. Calculate the absolute value of the centered Fourier transform of image I
    img_fft_abs = np.abs(img_fft_center)
    # 4. Calculate M where M is the maximum value of the frequency component in F
    img_fft_max = np.max(img_fft_abs)
    # 5. Calculate the total number of pixels in F whose pixel value > thres,
    # where thres = M / 1000
    img_th = np.count_nonzero(img_fft_abs > img_fft_max / 1000)
    # 6. Calculate image quality measure (FM)
    return img_th / np.multiply(*input_img.shape)

In [ ]:
def get_image_lap1_quality(input_img: np.ndarray) -> float:
    """Image quality measure LAP1 (Energy of Laplacian)"""
    lap_energy = np.pow(cv2.Laplacian(src=input_img, ddepth=cv2.CV_16S, ksize=3), 2)
    return np.sum(lap_energy) / np.max(lap_energy)

In [ ]:
def get_video_quality(
    input_video_frames: list[np.ndarray],
    quality_metric: Callable = get_image_fm_quality,
) -> list[float]:
    """Get video quality for each frame"""
    return [
        quality_metric(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        for frame in input_video_frames
    ]


def get_max_quality(video_quality: list[float]) -> int:
    """Get frame index with max FM image quality index"""
    return np.argmax(video_quality)

Herramienta para obtener el ROI de la imagen (region of interest):

In [ ]:
cartesian_tuple = tuple[int, int]  # typedef


def get_image_center_roi(
    height: int, width: int, roi_percentage: float
) -> tuple[cartesian_tuple, cartesian_tuple]:
    """Get a centered ROI coordinates for a given percentage of the image"""
    roi_width = np.sqrt(width * height * roi_percentage)
    roi_upper_left = int(width / 2 - roi_width / 2), int(height / 2 - roi_width / 2)
    roi_lower_right = int(width / 2 + roi_width / 2), int(height / 2 + roi_width / 2)
    return roi_upper_left, roi_lower_right


def get_image_matrix_roi(
    height: int, width: int, nrows: int, ncols: int, roi_percentage: float
) -> tuple[cartesian_tuple, cartesian_tuple]:
    roi_width = np.sqrt(width * height * roi_percentage)
    space_factor = 2.0
    roi_matrix = []
    for i in range(-int(ncols / 2), int(ncols / 2) + 1):
        for j in range(-int(nrows / 2), int(nrows / 2) + 1):
            roi_upper_left = int(width / 2 - roi_width / 2 + i * space_factor * roi_width), int(
                height / 2 - roi_width / 2 + j * space_factor * roi_width
            )
            roi_lower_right = int(width / 2 + roi_width / 2 + i * space_factor * roi_width), int(
                height / 2 + roi_width / 2 + j * space_factor * roi_width
            )
            roi_matrix.append((roi_upper_left, roi_lower_right))
    return roi_matrix


def get_image_from_roi(
    input_image: np.ndarray,
    roi_upper_left: cartesian_tuple,
    roi_lower_right: cartesian_tuple,
):
    return input_image[
        roi_upper_left[1] : roi_lower_right[1], roi_upper_left[0] : roi_lower_right[0]
    ]

Herramientas para anotar las imágenes:

In [ ]:
cartesian_tuple = tuple[int, int]  # typedef

FONT_SCALE = 1.8
FONT_FACE = cv2.FONT_HERSHEY_PLAIN
FONT_THICKNESS = 1


def set_annotation_fm_value(
    image: np.ndarray,
    fm_value: float,
    text_color: tuple[int, int, int] = (255, 255, 0),
):
    """Add FM metric text in the image top right corner"""
    cv2.putText(
        image,
        f"FM={fm_value:.4f}",
        (450, 50),
        fontFace=FONT_FACE,
        fontScale=FONT_SCALE,
        color=text_color,
        thickness=FONT_THICKNESS,
        lineType=cv2.LINE_AA,
    )


def set_annotation_roi(
    image: np.ndarray,
    upper_left: cartesian_tuple,
    lower_right: cartesian_tuple,
    roi_color: tuple[int, int, int] = (255, 255, 0),
):
    cv2.rectangle(
        image,
        upper_left,
        lower_right,
        color=roi_color,
        thickness=2,
        lineType=cv2.LINE_8,
    )

### Image sharpness measure applied to the complete frame

In [ ]:
video_frames = get_video_frames(input_video_path=VIDEO_FILE_PATH)
video_quality_fm = get_video_quality(video_frames, quality_metric=get_image_fm_quality)
frame_max_quality_fm = get_max_quality(video_quality_fm)
video_quality_lap1 = get_video_quality(
    video_frames, quality_metric=get_image_lap1_quality
)
frame_max_quality_lap1 = get_max_quality(video_quality_lap1)

fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(18, 3)

axs[0].set_title("Original video")
video_preview = axs[0].imshow(video_frames[0])

axs[1].set_title("Image quality (FM) on complete frame")
axs[1].axvline(frame_max_quality_fm, ls="-", color="g", lw=1, zorder=10)
plot_quality_fm = axs[1].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[1].plot([i for i in range(len(video_frames))], video_quality_fm)

axs[2].set_title("Image quality (LAP1) on complete frame")
axs[2].axvline(frame_max_quality_lap1, ls="-", color="g", lw=1, zorder=10)
plot_quality_lap1 = axs[2].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[2].plot([i for i in range(len(video_frames))], video_quality_lap1)

color_red = (255, 0, 0)
color_green = (0, 255, 0)


def update(i):
    annotated_frame = video_frames[i].copy()
    set_annotation_fm_value(
        annotated_frame,
        video_quality_fm[i],
        color_green if i == frame_max_quality_fm else color_red,
    )
    video_preview.set_data(annotated_frame)
    plot_quality_fm.set_xdata([i, i])
    plot_quality_lap1.set_xdata([i, i])
    return video_preview, plot_quality_fm, plot_quality_lap1


ani = animation.FuncAnimation(
    fig=fig, func=update, frames=len(video_frames), interval=30, repeat=False, blit=True
)

HTML(ani.to_jshtml())

### Image sharpness measure in Frequency Domain applied to a ROI

In [ ]:
video_frames = get_video_frames(input_video_path=VIDEO_FILE_PATH)
video_height, video_width, _ = video_frames[0].shape
pt1, pt2 = get_image_center_roi(video_height, video_width, 0.01)
roi_frames = [get_image_from_roi(frame, pt1, pt2) for frame in video_frames]
video_quality_fm = get_video_quality(roi_frames, quality_metric=get_image_fm_quality)
frame_max_quality_fm = get_max_quality(video_quality_fm)
video_quality_lap1 = get_video_quality(
    roi_frames, quality_metric=get_image_lap1_quality
)
frame_max_quality_lap1 = get_max_quality(video_quality_lap1)

fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(18, 3)

axs[0].set_title("Original video")
video_preview = axs[0].imshow(video_frames[0])

axs[1].set_title("Image quality (FM) on centered ROI")
axs[1].axvline(frame_max_quality_fm, ls="-", color="g", lw=1, zorder=10)
plot_quality_fm = axs[1].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[1].plot([i for i in range(len(video_frames))], video_quality_fm)

axs[2].set_title("Image quality (LAP1) on centered ROI")
axs[2].axvline(frame_max_quality_lap1, ls="-", color="g", lw=1, zorder=10)
plot_quality_lap1 = axs[2].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[2].plot([i for i in range(len(video_frames))], video_quality_lap1)

color_red = (255, 0, 0)
color_green = (0, 255, 0)


def update(i):
    annotated_frame = video_frames[i].copy()
    set_annotation_fm_value(
        annotated_frame,
        video_quality_fm[i],
        color_green if i == frame_max_quality_fm else color_red,
    )
    set_annotation_roi(
        annotated_frame,
        pt1,
        pt2,
        color_green if i == frame_max_quality_fm else color_red,
    )
    video_preview.set_data(annotated_frame)
    plot_quality_fm.set_xdata([i, i])
    plot_quality_lap1.set_xdata([i, i])
    return video_preview, plot_quality_fm, plot_quality_lap1


ani = animation.FuncAnimation(
    fig=fig, func=update, frames=len(video_frames), interval=30, repeat=False, blit=True
)

HTML(ani.to_jshtml())

### Image sharpness measure in Frequency Domain applied to a ROI matrix

In [ ]:
video_frames = get_video_frames(input_video_path=VIDEO_FILE_PATH)
video_height, video_width, _ = video_frames[0].shape

roi_matrix = get_image_matrix_roi(
    height=video_height, width=video_width, nrows=3, ncols=3, roi_percentage=0.001
)

video_quality_fm_roi = []
video_quality_lap1_roi = []
frame_max_quality_fm_roi = []
frame_max_quality_lap1_roi = []
for roi_upper_left, roi_lower_right in roi_matrix:
    roi_frames = [
        get_image_from_roi(frame, roi_upper_left, roi_lower_right)
        for frame in video_frames
    ]
    video_quality_fm_roi.append(
        get_video_quality(roi_frames, quality_metric=get_image_fm_quality)
    )
    frame_max_quality_fm_roi.append(get_max_quality(video_quality_fm_roi[-1]))
    video_quality_lap1_roi.append(
        get_video_quality(roi_frames, quality_metric=get_image_lap1_quality)
    )
    frame_max_quality_lap1_roi.append(get_max_quality(video_quality_lap1_roi[-1]))

# Get image quality average of the matrix
video_quality_fm_mean = [
    np.mean([roi_quality[frame_idx] for roi_quality in video_quality_fm_roi])
    for frame_idx in range(len(video_frames))
]
video_quality_lap1_mean = [
    np.mean([roi_quality[frame_idx] for roi_quality in video_quality_lap1_roi])
    for frame_idx in range(len(video_frames))
]

fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(18, 3)

axs[0].set_title("Original video")
video_preview = axs[0].imshow(video_frames[0])

axs[1].set_title("Image quality (FM) on ROI matrix (average)")
plot_quality_fm = axs[1].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[1].plot([i for i in range(len(video_frames))], video_quality_fm_mean)
axs[1].axvline(
    get_max_quality(video_quality_fm_mean), ls="-", color="g", lw=1, zorder=10
)

axs[2].set_title("Image quality (LAP1) on ROI matrix (average)")
plot_quality_lap1 = axs[2].axvline(0, ls="-", color="r", lw=1, zorder=10)
axs[2].plot([i for i in range(len(video_frames))], video_quality_lap1_mean)
axs[2].axvline(
    get_max_quality(video_quality_lap1_mean), ls="-", color="g", lw=1, zorder=10
)

color_red = (255, 0, 0)
color_green = (0, 255, 0)


def update(i):
    annotated_frame = video_frames[i].copy()
    for roi_idx, (roi_upper_left, roi_lower_right) in enumerate(roi_matrix):
        set_annotation_roi(
            annotated_frame,
            roi_upper_left,
            roi_lower_right,
            color_green if i == frame_max_quality_fm_roi[roi_idx] else color_red,
        )
    set_annotation_fm_value(
        annotated_frame,
        video_quality_fm[i],
        color_green if i in frame_max_quality_fm_roi else color_red,
    )
    video_preview.set_data(annotated_frame)
    plot_quality_fm.set_xdata([i, i])
    plot_quality_lap1.set_xdata([i, i])
    return video_preview, plot_quality_fm, plot_quality_lap1


ani = animation.FuncAnimation(
    fig=fig, func=update, frames=len(video_frames), interval=30, repeat=False, blit=True
)

HTML(ani.to_jshtml())